### CAFR10

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import torch.nn as nn
import torchvision
import torch.optim as optim
from datagen import *
from nets import *
from backdoor import Backdoor
from defense import Defense

# move to GPU (if applicable)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

/home/andmholm/Backdoor/backdoor_env/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Choosing Network and Dataset

In [2]:
# choose network configuration and dataset
backdoor = Backdoor(VggNet, config='16-layer', channels=3, classes=10, dropout=0.5)
backdoor.create_models(optim.SGD, nn.CrossEntropyLoss, lr=0.01)
backdoor.load_data(torchvision.datasets.CIFAR10, ImageEntity)

Files already downloaded and verified
Files already downloaded and verified


### Poison setup

In [3]:
# create patches, target labels, and define transformations/merges 
patches = ImagePatch((9, 9), 3, 'random').get_patches(len(backdoor))
targets = {i : i + 1 if i < 9 else 0 for i in range(10)}
transforms = (None, [RotateTransform()], [LabelTransform(targets)])
merge = ImageMerge(select=True)

### Poisoning

In [4]:
# poison 20% of the data
backdoor.poison(patches, transforms, merge, pct=0.2)

### Training Base and Trojan Net

In [5]:
# train the base DNN & the trojan DNN 
loss = backdoor.train(2, epochs=3, verbose=True, device=device, batch_size=16)

Training started
Epoch 1/3 | 24.99% | Loss: 1.8071 | Samples trained: 12496/50000
Epoch 1/3 | 49.98% | Loss: 1.6234 | Samples trained: 24992/50000
Epoch 1/3 | 74.98% | Loss: 1.4989 | Samples trained: 37488/50000
Epoch 1/3 | 99.97% | Loss: 1.4004 | Samples trained: 49984/50000
Epoch 1 complete | Loss: 1.4006
Epoch 2/3 | 24.99% | Loss: 0.9989 | Samples trained: 12496/50000
Epoch 2/3 | 49.98% | Loss: 0.9544 | Samples trained: 24992/50000
Epoch 2/3 | 74.98% | Loss: 0.9124 | Samples trained: 37488/50000
Epoch 2/3 | 99.97% | Loss: 0.8750 | Samples trained: 49984/50000
Epoch 2 complete | Loss: 0.8751
Epoch 3/3 | 24.99% | Loss: 0.7028 | Samples trained: 12496/50000
Epoch 3/3 | 49.98% | Loss: 0.6808 | Samples trained: 24992/50000
Epoch 3/3 | 74.98% | Loss: 0.6546 | Samples trained: 37488/50000
Epoch 3/3 | 99.97% | Loss: 0.6324 | Samples trained: 49984/50000
Epoch 3 complete | Loss: 0.6325
Training complete | Net Average Loss: 0.9694 | Total epochs: 3
Training started
Epoch 1/3 | 24.99% | Loss: 

### Evaluating Backdoor

In [6]:
# evulate backdoor based on VA & ASR
metrics = backdoor.eval(verbose=True, device=device)

Accuracy on Clean | Base 77.62% | Trojan 75.92% | Difference -1.70%
Base Accuracy on Poison 3.32% | Attack Success Rate (ASR): 65.02%
Average Tensor Distance: 305.93 | Net Tensor Difference 15296582.63


### Creating a Defense

In [7]:
# init the defense
defense = Defense(backdoor)

### Detection setup

In [8]:
# get base & trojan DNN, both clean & poisoned entitysets, sizes ranges to create synthetic patch
base, trojan = backdoor.get_net_modules()
cleantrain, poisontrain, cleantest, poisontest = backdoor.get_datasets()
patch_size_ranges = [(3, 3), (5, 5), (7, 7), (9, 9), (11, 11)]

### Detection Base

In [9]:
# detect backdoor with 30% of the clean samples poisoned and detect based on a 10% drop in VA for base DNN
metrics = defense.detect(base, cleantest, threshold=0.1, size_ranges=patch_size_ranges,  pct=0.3, verbose=True, device=device)

Patch size (3, 3) | Synthetic Poison Accuracy 77.25% | Original Accuracy 77.62% | Difference -0.37%
Patch size (5, 5) | Synthetic Poison Accuracy 76.50% | Original Accuracy 77.62% | Difference -1.12%
Patch size (7, 7) | Synthetic Poison Accuracy 75.59% | Original Accuracy 77.62% | Difference -2.03%
Patch size (9, 9) | Synthetic Poison Accuracy 75.05% | Original Accuracy 77.62% | Difference -2.57%
Patch size (11, 11) | Synthetic Poison Accuracy 72.75% | Original Accuracy 77.62% | Difference -4.87%
Average Accuracy 75.43% | Average Difference -2.19% | Lowest Score: 72.75% | Likihood of Backdoor: Low


### Detection Trojan

In [10]:
# detect backdoor with 30% of the clean samples poisoned and detect based on a 10% drop in VA for trojan DNN
metrics = defense.detect(trojan, cleantest, threshold=0.1, size_ranges=patch_size_ranges,  pct=0.3, verbose=True, device=device)

Patch size (3, 3) | Synthetic Poison Accuracy 75.88% | Original Accuracy 75.92% | Difference -0.04%
Patch size (5, 5) | Synthetic Poison Accuracy 75.27% | Original Accuracy 75.92% | Difference -0.65%
Patch size (7, 7) | Synthetic Poison Accuracy 73.75% | Original Accuracy 75.92% | Difference -2.17%
Patch size (9, 9) | Synthetic Poison Accuracy 66.09% | Original Accuracy 75.92% | Difference -9.83%
Patch size (11, 11) | Synthetic Poison Accuracy 54.48% | Original Accuracy 75.92% | Difference -21.44%
Average Accuracy 69.09% | Average Difference -6.83% | Lowest Score: 54.48% | Likihood of Backdoor: High


### Blocking Patches setup

In [11]:
# reverse labels (for research VA purposes), create patch to compare variances with in parts of images (same size as patch), define samples to block
labels = {i : i - 1 if i > 0 else 9 for i in range(10)}
patch = ImagePatch((10, 10), 3, 'random')
n = 5000

### Blocking

In [12]:
# block poisoned images with average rgb values the same dimensions as the patch
blockedloader = defense.block(poisontest, patch, labels, n)

### Testing after Block Base

In [13]:
# retest after blocking for base DNN
metrics = defense.test(base, blockedloader, verbose=True, device=device)

Testing started
24.72% Testing complete | Loss: 1.1140 | Accuracy: 0.6179
49.44% Testing complete | Loss: 1.1390 | Accuracy: 0.6179
74.16% Testing complete | Loss: 1.1456 | Accuracy: 0.6198
98.88% Testing complete | Loss: 1.1340 | Accuracy: 0.6328
Testing complete | Loss: 1.1298 | Accuracy: 63.48%


### Testing after Block Trojan

In [14]:
# retest after blocking for trojan DNN
metrics = defense.test(trojan, blockedloader, verbose=True, device=device)

Testing started
24.72% Testing complete | Loss: 2.0480 | Accuracy: 0.4432
49.44% Testing complete | Loss: 2.0900 | Accuracy: 0.4325
74.16% Testing complete | Loss: 2.1359 | Accuracy: 0.4223
98.88% Testing complete | Loss: 2.1421 | Accuracy: 0.4325
Testing complete | Loss: 2.1377 | Accuracy: 43.43%


### Defense by retraining

In [17]:
# get a clean dataloader, reset the defense net module, retrain the defense net model (og trojan model)
dataloader = cleantrain.get_dataloader()
defense.reset()
loss = defense.retrain(dataloader, epochs=3, verbose=True, device=device, batch_size=16)

Training started
Epoch 1/3 | 24.95% | Loss: 0.6534 | Samples trained: 12480/50000
Epoch 1/3 | 49.90% | Loss: 0.6448 | Samples trained: 24960/50000
Epoch 1/3 | 74.86% | Loss: 0.6308 | Samples trained: 37440/50000
Epoch 1/3 | 99.81% | Loss: 0.5981 | Samples trained: 49920/50000
Epoch 1 complete | Loss: 0.5976
Epoch 2/3 | 24.95% | Loss: 0.6508 | Samples trained: 12480/50000
Epoch 2/3 | 49.90% | Loss: 0.6457 | Samples trained: 24960/50000
Epoch 2/3 | 74.86% | Loss: 0.6305 | Samples trained: 37440/50000
Epoch 2/3 | 99.81% | Loss: 0.5977 | Samples trained: 49920/50000
Epoch 2 complete | Loss: 0.5972
Epoch 3/3 | 24.95% | Loss: 0.6536 | Samples trained: 12480/50000
Epoch 3/3 | 49.90% | Loss: 0.6468 | Samples trained: 24960/50000
Epoch 3/3 | 74.86% | Loss: 0.6311 | Samples trained: 37440/50000
Epoch 3/3 | 99.81% | Loss: 0.5968 | Samples trained: 49920/50000
Epoch 3 complete | Loss: 0.5963
Training complete | Net Average Loss: 0.5970 | Total epochs: 3


### Evaluation after retraining

In [18]:
# re-evaluate the trojan DNN
cleanloader, poisonloader = cleantest.get_dataloader(), poisontest.get_dataloader()
metrics = defense.eval(cleanloader, poisonloader, verbose=True, device=device)

Accuracy on clean | Base 77.62% | Trojan 75.92% | Defense 74.86%
Accuracy on Posion | Base 3.32% | Defense 60.07% | Trojan ASR 65.02%
Difference from Baseline | Trojan -1.70% | Defense -2.76%
Defense Effectiveness | 4.95% decrease in ASR


In [ ]:
defense.view_named_modules()

### Pruning setup

In [20]:
# determine layers to prune, decide percent of neurons to prune, reset the defense (original trojan DNN)
layers = ['layers.28', 'layers.26', 'layers.24', 'layers.21']
amount = 0.5
defense.reset()

### Pruning based on l-Infinity norm of weights from layers

In [21]:
# prune 50% the weights of last 4 convolutional layers based on the l-Infinity norm
defense.prune(layers, amount)

### Evaluation after prune

In [22]:
# evaluate the model after pruning
metrics = defense.eval(cleanloader, poisonloader, verbose=True, device=device)

Accuracy on clean | Base 77.62% | Trojan 75.92% | Defense 15.84%
Accuracy on Posion | Base 3.32% | Defense 13.59% | Trojan ASR 65.02%
Difference from Baseline | Trojan -1.70% | Defense -61.78%
Defense Effectiveness | 51.43% decrease in ASR


### Retraining to recover

In [23]:
# retrain to recover
loss = defense.retrain(dataloader, epochs=3, verbose=True, device=device, batch_size=16)

Training started
Epoch 1/3 | 24.95% | Loss: 1.5157 | Samples trained: 12480/50000
Epoch 1/3 | 49.90% | Loss: 1.5131 | Samples trained: 24960/50000
Epoch 1/3 | 74.86% | Loss: 1.5112 | Samples trained: 37440/50000
Epoch 1/3 | 99.81% | Loss: 1.5020 | Samples trained: 49920/50000
Epoch 1 complete | Loss: 1.5018
Epoch 2/3 | 24.95% | Loss: 1.5133 | Samples trained: 12480/50000
Epoch 2/3 | 49.90% | Loss: 1.5119 | Samples trained: 24960/50000
Epoch 2/3 | 74.86% | Loss: 1.5102 | Samples trained: 37440/50000
Epoch 2/3 | 99.81% | Loss: 1.5010 | Samples trained: 49920/50000
Epoch 2 complete | Loss: 1.5011
Epoch 3/3 | 24.95% | Loss: 1.5174 | Samples trained: 12480/50000
Epoch 3/3 | 49.90% | Loss: 1.5148 | Samples trained: 24960/50000
Epoch 3/3 | 74.86% | Loss: 1.5099 | Samples trained: 37440/50000
Epoch 3/3 | 99.81% | Loss: 1.5014 | Samples trained: 49920/50000
Epoch 3 complete | Loss: 1.5013
Training complete | Net Average Loss: 1.5014 | Total epochs: 3


### Evaluation after retraining

In [24]:
metrics = defense.eval(cleanloader, poisonloader, verbose=True, device=device)

Accuracy on clean | Base 77.62% | Trojan 75.92% | Defense 65.45%
Accuracy on Posion | Base 3.32% | Defense 34.55% | Trojan ASR 65.02%
Difference from Baseline | Trojan -1.70% | Defense -12.17%
Defense Effectiveness | 30.47% decrease in ASR
